In [2]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
appName("Sneha Spark Session").\
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [1]:
spark

NameError: name 'spark' is not defined

In [61]:
order_schema = 'order_id long, order_date string, customer_id long, order_status string'

In [62]:
order_df = spark.read.\
format("csv").\
schema(order_schema).\
load("/public/trendytech/orders/orders_1gb.csv")

In [63]:
order_df.show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|       1837|         CLOSED|
|      13|2013-07-25 00:00:...|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:...|       98

In [64]:
order_df = spark.read.\
format("csv").\
schema(order_schema).\
load("/user/itv017244/sparkwriterdemo1")

In [65]:
order_df.createOrReplaceTempView("orders")

In [66]:
spark.sql("select count(*) from orders where order_status ='CLOSED' ")

count(1)
2833500


In [67]:
order_df.write.format('csv').mode('overwrite').partitionBy('order_status').option('path','/user/itv017244/partition_demo_output1').save()

In [68]:
order_df.rdd.getNumPartitions()

9

In [69]:
spark.sql("select distinct(order_status) from orders").count()

9

In [31]:
order_df = spark.read.\
format("csv").\
schema(order_schema).\
load("/user/itv017244/partition_demo_output1")

In [32]:
order_df.createOrReplaceTempView("orders")

In [33]:
spark.sql("select count(*) from orders where order_status ='CLOSED' ")

count(1)
2833500


In [34]:
spark.sql("select count(*) from orders where customer_id =256 ")

count(1)
3750


In [36]:
spark.sql("select count(*) from orders where order_status ='CLOSED' and customer_id = 256 ")

count(1)
750


In [37]:
spark.sql("select count(*) from orders where customer_id = 256  and  order_status ='CLOSED'")

count(1)
750


In [40]:
customers_df = spark.read.format('csv').option('inferSchema',True).load('/public/trendytech/retail_db/customers/part-00000')

In [41]:
customers_df.show()

+---+-----------+---------+---------+---------+--------------------+-------------+---+-----+
|_c0|        _c1|      _c2|      _c3|      _c4|                 _c5|          _c6|_c7|  _c8|
+---+-----------+---------+---------+---------+--------------------+-------------+---+-----+
|  1|    Richard|Hernandez|XXXXXXXXX|XXXXXXXXX|  6303 Heather Plaza|  Brownsville| TX|78521|
|  2|       Mary|  Barrett|XXXXXXXXX|XXXXXXXXX|9526 Noble Embers...|    Littleton| CO|80126|
|  3|        Ann|    Smith|XXXXXXXXX|XXXXXXXXX|3422 Blue Pioneer...|       Caguas| PR|  725|
|  4|       Mary|    Jones|XXXXXXXXX|XXXXXXXXX|  8324 Little Common|   San Marcos| CA|92069|
|  5|     Robert|   Hudson|XXXXXXXXX|XXXXXXXXX|10 Crystal River ...|       Caguas| PR|  725|
|  6|       Mary|    Smith|XXXXXXXXX|XXXXXXXXX|3151 Sleepy Quail...|      Passaic| NJ| 7055|
|  7|    Melissa|   Wilcox|XXXXXXXXX|XXXXXXXXX|9453 High Concession|       Caguas| PR|  725|
|  8|      Megan|    Smith|XXXXXXXXX|XXXXXXXXX|3047 Foggy Forest...|  

In [44]:
customers_new_df = customers_df.toDF("customer_id","customer_fname","customer_lastname","customer_email","customer_password","customer_street","customer_city","customer_state","customer_zipcode")

In [45]:
customers_new_df.show()

+-----------+--------------+-----------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lastname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+-----------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|        Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|          Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|            Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|             725|
|          4|          Mary|            Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|    

In [49]:
customers_new_df.write.format('parquet').mode('overwrite').partitionBy("customer_state","customer_city").option('path','/user/itv017244/partition_demo_output2').save()

In [50]:
customers_df = spark.read.format('parquet')\
.load('/user/itv017244/partition_demo_output2')

In [51]:
customers_df.show()

+-----------+--------------+-----------------+--------------+-----------------+--------------------+----------------+--------------+-------------+
|customer_id|customer_fname|customer_lastname|customer_email|customer_password|     customer_street|customer_zipcode|customer_state|customer_city|
+-----------+--------------+-----------------+--------------+-----------------+--------------------+----------------+--------------+-------------+
|          3|           Ann|            Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|             725|            PR|       Caguas|
|          5|        Robert|           Hudson|     XXXXXXXXX|        XXXXXXXXX|10 Crystal River ...|             725|            PR|       Caguas|
|          7|       Melissa|           Wilcox|     XXXXXXXXX|        XXXXXXXXX|9453 High Concession|             725|            PR|       Caguas|
|          9|          Mary|            Perez|     XXXXXXXXX|        XXXXXXXXX| 3616 Quaking Street|             725| 

In [54]:
customers_new_df.rdd.getNumPartitions()

1

In [55]:
customers_df.createOrReplaceTempView("customers")

In [57]:
spark.sql("select * from customers where customer_id = 19").show()

Py4JJavaError: An error occurred while calling o410.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Thread.java:750)